In [4]:
import os
import pandas as pd
import numpy as np
import scipy.io
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS

class BLP(object):
    '''
    Jia Yan
    02/21/2023
    '''
    
    def __init__(self, path_data, file_data, path_str_data, file_str_data, ndraws=500, tol_fp=1e-12):
        # set parameters
        self.ndraws = ndraws
        self.tol_fp = tol_fp
        
        # create data
        data = scipy.io.loadmat(os.path.join(path_data, file_data))
        model_name = scipy.io.loadmat(os.path.join(path_str_data, file_str_data))['model_name']
        v_list = ['outshr', 'const', 'mpd', 'mpg', 'air', 'space', 'hpwt', 'price', 'trend']
        self.nmarkets = data['trend'].max() + 1
        self.df = data['share']
        for item in v_list:
            self.df = np.concatenate([self.df, data[item]], axis=1)
        self.df = pd.DataFrame(self.df, columns = ['share'] + v_list)
        self.df['model_name'] = model_name
        self.df['maker'] = self.df['model_name'].transform(lambda x: x[0:2])
        
        # demand model variables
        self.attributes = ['const', 'mpd', 'air', 'space', 'hpwt', 'price']
        
        # demand model variables with random coefficient
        self.attributes_random = ['const', 'mpd', 'air', 'space', 'hpwt']
        
        # marginal cost variables
        self.mc = ['const', 'mpg', 'air', 'space', 'hpwt', 'trend']
        self.Xmat = self.df[self.attributes].to_numpy()
        self.Mcmat = self.df[self.mc].copy()
        self.Mcmat['mpg'] = np.log(self.Mcmat['mpg'])
        self.Mcmat['space'] = np.log(self.Mcmat['space'])
        self.Mcmat['hpwt'] = np.log(self.Mcmat['hpwt'])
        self.Mcmat = self.Mcmat.to_numpy()
        
        '''
        Take standard normal draws for approximating integrals in market share and mark-up computations.
        Better to use halton draws
        '''
        self.draws = np.random.randn(self.nmarkets, self.ndraws, len(self.attributes_random))    
        
        '''
        creat instruments for price: sum of attributes of rival products
        '''
        z_list = ['const']
        self.IV_list = ['const', 'mpd', 'air', 'space', 'hpwt'] # the first part of IV are exogenous regressors
        for var in z_list:
            name_own = var + "_" + "z" + "_" + "own"
            self.IV_list.append(name_own)
            name_rival = var + "_" + "z" + "_" + "rival"
            self.IV_list.append(name_rival)
            
            self.df[name_own] = self.df.groupby(['trend', 'maker'])[var].transform(lambda x: x.sum())
            self.df[name_own] = self.df[name_own] - self.df[var]
            
            self.df['junk']= self.df.groupby(['trend'])[var].transform(lambda x: x.sum()) - self.df[var]
            self.df[name_rival] = self.df['junk'] - self.df[name_own]
        
        self.Zmat = self.df[self.IV_list].to_numpy()
        self.weight_mat = np.linalg.inv(np.matmul(np.transpose(self.Zmat), self.Zmat)) # weighting matrix in GMM estimation
        
        # creat projection matrix of instruments for future computations 
        pz = np.matmul(self.Zmat, self.weight_mat)
        pz = np.matmul(pz, np.transpose(self.Zmat))
        self.project_mat = np.matmul(np.transpose(self.Xmat), pz)
        self.project_mat = np.matmul(self.project_mat, self.Xmat)
        self.project_mat = np.linalg.inv(self.project_mat)
        self.project_mat = np.matmul(self.project_mat, np.transpose(self.Xmat))
        self.project_mat = np.matmul(self.project_mat, pz)
        
        # finally, dependent variable in demand models without random coefficients
        self.y_fixed = np.log(self.df['share']/self.df['outshr'])
        
    def ols(self):
        '''
        replicate the first column of table 3
        '''
        y = np.log(self.df['share']/self.df['outshr'])
        #b = np.matmul(np.transpose(self.Xmat), self.Xmat)
        #b = np.linalg.inv(b)
        #b = np.matmul(b, np.transpose(self.Xmat))
        #return np.matmul(b, self.y_fixed)
        return sm.OLS(self.y_fixed, self.Xmat).fit()
        
    def iv(self):
        '''
        replicate the second column of table 3
        '''
        #return np.matmul(self.project_mat, self.y_fixed)
        return IV2SLS(self.y_fixed, self.Xmat, self.Zmat).fit()
    
    def hedonic_price(self):
        '''replicate the third column of table 3'''
        y = np.log(self.df['price'])
        return sm.OLS(y, self.Mcmat).fit()
    
    def market_share(self, mid, delta, xv):
        draws = self.draws[mid]
        s = np.zeros(len(delta))
        for r in range(self.ndraws):
            w = draws[r]
            v = np.exp(delta + (w * xv).sum(axis=1))
            s = s + (v / (1 + np.sum(v)))
        return (1/self.ndraws) * s 
    
    def fixed_point(self, pack):
        mid = pack['mid']
        df = pack['df']
        sigmas = pack['sigmas']
        s0 = df['share'].to_numpy()
        xv = sigmas * df[self.attributes_random].to_numpy()
        check = 1.0
        delta_ini = np.zeros(len(s0))
        while check > self.tol_fp:
            
            delta_new = delta_ini + (np.log(s0) - np.log(self.market_share(mid, delta_ini, xv)))
            check = np.max(abs(delta_new - delta_ini))
            delta_ini = delta_new
        return delta_new
        
    def mean_utility(self,sigmas):
        """
        sigmas: an 1_D array with the shape (len(self.attributes_random), ), which contains
        the standard errors of random coefficients
        """
        df = self.df.copy()
        v_list = ['share'] + self.attributes_random
        
        '''
        # step 1: solve mean utility (delta_j) from the fixed-point iteration
        '''
        df_list = [{'mid': int(mid), 'df': d[v_list], 'sigmas': sigmas} for mid, d in df.groupby(['trend'])]
        delta_j = tuple(map(self.fixed_point, df_list))
        delta_j = np.concatenate(delta_j, axis=0) # an array with the shape(2217,)
        
        '''
        step 2: uncover mean part of coefficients (beta_bar) from delta_j, which is equivalent to 
        running an IV estimation using delta_j as the dependent variable
        '''
        beta_bar = np.matmul(self.project_mat, delta_j) 
        
        '''
        step 3: uncover ommited product attributes (xi_j) from delta_j and beta_bar
        '''
        xi_j = delta_j - np.matmul(self.Xmat, beta_bar)
        
        return {'beta_bar': beta_bar, 'xi_j': xi_j} 
    
    def GMM_obj(self, sigmas):
        
        '''
        step 4: interact xi_j with instruments,which include exogenous regressors (veihicles' own
        exogenous attributes) and instruments for price (sum of attributes of competing products)
        '''
        xi_j = self.mean_utility(sigmas)['xi_j']
        moments = np.matmul(np.transpose(self.Zmat), xi_j) # an array with the shape (m, ), where m is the number of IVs
        
        '''
        step 5: compute the GMM objective function
        '''
        f = np.matmul(moments, self.weight_mat)
        f = (1/len(self.df)) * np.matmul(f, moments)
        return f
    
    def optimization(self, objfun, para):
        '''
        Parameters
        ----------
        objfun : a user defined objective function of para
            
        para : a 1-D array with the shape (k,), where k is the number of parameters.
        Returns
        -------
        dict
            A dictionary containing estimation results      
        '''
        v = opt.minimize(objfun, x0=para, jac=None, method='BFGS', 
                          options={'maxiter': 1000, 'disp': True})  
        return {'obj':v.fun, "Coefficients": v.x}

if __name__ == "__main__":
    
    #blp = BLP("/kaggle/input/blp-data/", "BLP_data.mat", "/kaggle/input/blp-str-data/", "BLP_data_str.mat")
    blp = BLP("/Users/otten/Downloads","BLP_data.mat","/Users/otten/Downloads","BLP_data_str.mat")
    pini = np.ones(len(blp.attributes_random)) * 0.2
    x = blp.GMM_obj(pini)
    beta_ols = blp.ols()
    beta_iv = blp.iv()
    beta_hedonic= blp.hedonic_price()
    print(beta_ols.summary())
    print(beta_iv.summary())
    print(sm.OLS(blp.df['price'], blp.Zmat).fit().summary()) # first-stage regression in IV estimation
    print(beta_hedonic.summary())
    
    

[0.01074215 0.01070843 0.01067493 0.01065552 0.01067533 0.01065906
 0.01068799 0.01076677 0.0107482  0.01074321 0.01074378 0.01074248
 0.01076568 0.01073854 0.01075208 0.01068667 0.01066583 0.01068624
 0.01074193 0.01067131 0.01067131 0.01067237 0.01073028 0.01082253
 0.01073294 0.01076271 0.0107573  0.01081883 0.01067799 0.01066344
 0.01066441 0.01067293 0.01067742 0.01073552 0.01074939 0.01069171
 0.01066404 0.01068578 0.01066385 0.01066509 0.01072354 0.01078218
 0.01074991 0.01074306 0.01068881 0.01066179 0.0107497  0.01071221
 0.01070427 0.01065796 0.01069206 0.01073604 0.01075726 0.01067949
 0.01067157 0.01066503 0.01067187 0.01072007 0.01066295 0.01068357
 0.01072428 0.01073409 0.01075821 0.01076301 0.01071224 0.01077193
 0.01097622 0.01084516 0.01098967 0.01081125 0.01097763 0.01085864
 0.01068313 0.01068031 0.01072644 0.01076637 0.01084768 0.01071361
 0.0107934  0.01080069 0.01083798 0.01072257 0.01089332 0.01082638
 0.01078303 0.01068863 0.01079949 0.01089644 0.01078244 0.0106